In [ ]:
import numpy as np
import cv2 as cv
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import time

from lac.util import color_mask
from lac.perception.segmentation import Segmentation

%load_ext autoreload
%autoreload 2

# Hazard detection

Use combination of instance segmentation and depth estimation to identify hazards (rocks)


In [ ]:
segmentation = Segmentation()

In [ ]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/hazard_2")
# data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/data_collection_1")

i = 1001
FL_gray = cv.imread(os.path.join(data_path, "front_left", f"{i}.png"), cv.IMREAD_GRAYSCALE)
plt.imshow(FL_gray, cmap="gray")
plt.show()

In [ ]:
results, mask = segmentation.segment_rocks(Image.fromarray(FL_gray).convert("RGB"))

In [ ]:
FL_rgb = cv.cvtColor(FL_gray, cv.COLOR_GRAY2BGR)
mask_colored = color_mask(mask, (1, 0, 0)).astype(FL_rgb.dtype)
overlay = cv.addWeighted(FL_rgb, 1.0, mask_colored, beta=0.5, gamma=0)
plt.imshow(overlay)

In [1]:
from lac.perception.depth import DepthAnything

In [3]:
depth_model = DepthAnything()

In [3]:
import torch
from transformers import pipeline

checkpoint = "depth-anything/Depth-Anything-V2-base-hf"
pipe = pipeline("depth-estimation", model=checkpoint, device="cuda")

# # Force model outputs to use float32
# pipe.model.to(torch.float32)
pass

In [6]:
image = Image.open(os.path.join(data_path, "front_left", f"{i}.png"))

In [ ]:
pipe(image)

In [18]:
import os
from PIL import Image
from transformers import pipeline
import torch
import matplotlib.pyplot as plt

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = "depth-anything/Depth-Anything-V2-base-hf"
pipe = pipeline("depth-estimation", model=checkpoint, device=device)

In [ ]:
i = 100

image_path = os.path.expanduser(
    "~/LunarAutonomyChallenge/output/data_collection_1/front_left/{}.png".format(i)
)
image_depth_in = Image.open(image_path)

predictions = pipe(image_depth_in)

# Plot image and predicted depth side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={"wspace": 0, "hspace": 0})
axes[0].imshow(image_depth_in, cmap="gray")
axes[1].imshow(predictions["depth"], cmap="gray")
for ax in axes:
    ax.axis("off")
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
image = Image.open(os.path.join(data_path, "front_left", f"{i}.png"))
predictions = pipe(image)

# Plot image and predicted depth side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={"wspace": 0, "hspace": 0})
axes[0].imshow(image, cmap="gray")
axes[1].imshow(predictions["depth"], cmap="gray")
for ax in axes:
    ax.axis("off")
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()